In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import json
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
housing_df = pd.read_csv("Resources\DC_Properties.csv")

C:\Users\djhon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
housing_df['YEAR'] = pd.DatetimeIndex(housing_df['SALEDATE']).year

In [4]:
#Filter orginal dataframe to get to meaningful data 54,752 rows
hdf2 = housing_df.loc[housing_df['YEAR']== 2017, :]
hdf3 = hdf2.loc[hdf2['SOURCE'] == 'Residential', : ]
hdf4 = hdf3.dropna(subset=['PRICE','QUADRANT','WARD','FULLADDRESS','ZIPCODE','BEDRM','BATHRM','KITCHENS'])
hdf5 = hdf4[['QUADRANT','FULLADDRESS','ZIPCODE','BEDRM','BATHRM','KITCHENS','PRICE']]
hdf6 = hdf5.loc[hdf5['PRICE'] > 100000, :]
hdtable = pd.DataFrame(hdf6)

In [5]:
hdtable.to_csv("hdtable.csv", index=False, header=True)

In [6]:
# Create Engine
engine = create_engine("sqlite:///housingdata.sqlite")
# Declare a Base
Base = declarative_base()

In [7]:
# Define the ORM class for `Measurements`
class Measurement(Base):
    
    __tablename__ = 'hdata'

    id = Column(Integer, primary_key=True)
    QUADRANT = Column(String)
    FULLADDRESS = Column(String)
    ZIPCODE = Column(Integer)
    BEDRM = Column(Integer)
    BATHRM = Column(Integer)
    KITCHENS = Column(Integer)
    PRICE = Column(Float)

In [8]:
# Use `create_all` to create the tables
Base.metadata.create_all(engine)

In [9]:
# Use Pandas to Bulk insert each CSV file into their appropriate table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)

In [10]:
populate_table(engine, Measurement.__table__, 'hdtable.csv')

In [11]:
hadatadict = engine.execute("SELECT * FROM hdata LIMIT 100").fetchall()

In [12]:
df = pd.DataFrame(hadatadict, columns = ['Index','Quad', 'Address','Zipcode','Bedroom','Bathroom','Kitchens','Price'])

In [13]:
hdata = df.to_dict(orient='records')

In [14]:
with open("hdata1.js", "w") as f:
    json.dump(hdata,f,indent=4,)